# Recurrent Neural Networks

Recurrent Neural Networks (RNN) are very effective for Natural Language Processing and other sequence tasks because they have "memory". They can read inputs $x^{\langle t \rangle}$ (such as words) one at a time, and remember some information/context through the hidden layer activations that get passed from one time-step to the next. This allows a uni-directional RNN to take information from the past to process later inputs. A bidirection RNN can take context from both the past and the future. 

**Notation**:
- Superscript $[l]$ denotes an object associated with the $l^{th}$ layer. 
    - Example: $a^{[4]}$ is the $4^{th}$ layer activation. $W^{[5]}$ and $b^{[5]}$ are the $5^{th}$ layer parameters.

- Superscript $(i)$ denotes an object associated with the $i^{th}$ example. 
    - Example: $x^{(i)}$ is the $i^{th}$ training example input.

- Superscript $\langle t \rangle$ denotes an object at the $t^{th}$ time-step. 
    - Example: $x^{\langle t \rangle}$ is the input x at the $t^{th}$ time-step. $x^{(i)\langle t \rangle}$ is the input at the $t^{th}$ timestep of example $i$.
    
- Lowerscript $i$ denotes the $i^{th}$ entry of a vector.
    - Example: $a^{[l]}_i$ denotes the $i^{th}$ entry of the activations in layer $l$.



## 1 - Forward propagation for the basic Recurrent Neural Network
 
The basic RNN that you will implement has the structure below. In this example, $T_x = T_y$. 

<img src="https://imgur.com/Yaa79IN.png" style="width:500;height:300px;">
<caption><center> **Figure 1**: Basic RNN model </center></caption>

Here's how you can implement an RNN: 

**Code Instructions**:
1. Implement the calculations needed for one time-step of the RNN.
2. Implement a loop over $T_x$ time-steps in order to process all the inputs, one at a time. 

Let's go!

## 1.1 - RNN cell

A Recurrent neural network can be seen as the repetition of a single cell. You are first going to implement the computations for a single time-step. The following figure describes the operations for a single time-step of an RNN cell. 

<img src="https://imgur.com/vGxAY57.png" style="width:700px;height:300px;">
<caption><center> **Figure 2**: Basic RNN cell. Takes as input $x^{\langle t \rangle}$ (current input) and $a^{\langle t - 1\rangle}$ (previous hidden state containing information from the past), and outputs $a^{\langle t \rangle}$ which is given to the next RNN cell and also used to predict $y^{\langle t \rangle}$ </center></caption>


**Code Instructions**:
1. Compute the hidden state with tanh activation: $a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$.
2. Using your new hidden state $a^{\langle t \rangle}$, compute the prediction $\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$. We provided you a function: `softmax`.
3. Store $(a^{\langle t \rangle}, a^{\langle t-1 \rangle}, x^{\langle t \rangle}, parameters)$ in cache
4. Return $a^{\langle t \rangle}$ , $y^{\langle t \rangle}$ and cache

We will vectorize over $m$ examples. Thus, $x^{\langle t \rangle}$ will have dimension $(n_x,m)$, and $a^{\langle t \rangle}$ will have dimension $(n_a,m)$. 

## 1.2 - RNN forward pass 

RNN as the repetition of the cell you've just built. If your input sequence of data is carried over 10 time steps, then you will copy the RNN cell 10 times. Each cell tak
es as input the hidden state from the previous cell ($a^{\langle t-1 \rangle}$) and the current time-step's input data ($x^{\langle t \rangle}$). It outputs a hidden state ($a^{\langle t \rangle}$) and a prediction ($y^{\langle t \rangle}$) for this time-step.


<img src="https://imgur.com/YdNCgkN.png" style="width:800px;height:300px;">
<caption><center> **Figure 3**: Basic RNN. The input sequence $x = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$  is carried over $T_x$ time steps. The network outputs $y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$. </center></caption>

**Code Instructions**:
1. Create a vector of zeros ($a$) that will store all the hidden states computed by the RNN.
2. Initialize the "next" hidden state as $a_0$ (initial hidden state).
3. Start looping over each time step, your incremental index is $t$ :
    - Update the "next" hidden state and the cache by running `rnn_cell_forward`
    - Store the "next" hidden state in $a$ ($t^{th}$ position) 
    - Store the prediction in y
    - Add the cache to the list of caches
4. Return $a$, $y$ and caches


### 1.3 - Basic RNN  backward pass

We will start by computing the backward pass for the basic RNN-cell.

<img src="https://imgur.com/3EniMu4.png" style="width:500;height:300px;"> <br>
<caption><center> **Figure 4**: RNN-cell's backward pass. Just like in a fully-connected neural network, the derivative of the cost function $J$ backpropagates through the RNN by following the chain-rule from calculus. The chain-rule is also used to calculate $(\frac{\partial J}{\partial W_{ax}},\frac{\partial J}{\partial W_{aa}},\frac{\partial J}{\partial b})$ to update the parameters $(W_{ax}, W_{aa}, b_a)$. </center></caption>

#### Deriving the one step backward functions: 

To compute the `rnn_cell_backward` you need to compute the following equations. It is a good exercise to derive them by hand. 

The derivative of $\tanh$ is $1-\tanh(x)^2$.

Similarly for $\frac{ \partial a^{\langle t \rangle} } {\partial W_{ax}}, \frac{ \partial a^{\langle t \rangle} } {\partial W_{aa}},  \frac{ \partial a^{\langle t \rangle} } {\partial b}$, the derivative of  $\tanh(u)$ is $(1-\tanh(u)^2)du$. 

The final two equations also follow same rule and are derived using the $\tanh$ derivative. Note that the arrangement is done in a way to get the same dimensions to match.

#### Backward pass through the RNN

Computing the gradients of the cost with respect to $a^{\langle t \rangle}$ at every time-step $t$ is useful because it is what helps the gradient backpropagate to the previous RNN-cell. To do so, you need to iterate through all the time steps starting at the end, and at each step, you increment the overall $db_a$, $dW_{aa}$, $dW_{ax}$ and you store $dx$.

**Instructions**:

Implement the `rnn_backward` function. Initialize the return variables with zeros first and then loop through all the time steps while calling the `rnn_cell_backward` at each time timestep, update the other variables accordingly.

In the next part, you will build a more complex LSTM model, which is better at addressing vanishing gradients. The LSTM will be better able to remember a piece of information and keep it saved for many timesteps. 

## 2 - Long Short-Term Memory (LSTM) network

This following figure shows the operations of an LSTM-cell.

<img src="https://imgur.com/wRyYVQ6.png" style="width:500;height:400px;">
<caption><center> **Figure 4**: LSTM-cell. This tracks and updates a "cell state" or memory variable $c^{\langle t \rangle}$ at every time-step, which can be different from $a^{\langle t \rangle}$. </center></caption>

Similar to the RNN example above, you will start by understanding the LSTM cell for a single time-step. Then you can iteratively call it from inside a for-loop to have it process an input with $T_x$ time-steps. 

### About the gates

#### - Forget gate

For the sake of this illustration, lets assume we are reading words in a piece of text, and want use an LSTM to keep track of grammatical structures, such as whether the subject is singular or plural. If the subject changes from a singular word to a plural word, we need to find a way to get rid of our previously stored memory value of the singular/plural state. In an LSTM, the forget gate lets us do this: 

$$\Gamma_f^{\langle t \rangle} = \sigma(W_f[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_f)\tag{1} $$

Here, $W_f$ are weights that govern the forget gate's behavior. We concatenate $[a^{\langle t-1 \rangle}, x^{\langle t \rangle}]$ and multiply by $W_f$. The equation above results in a vector $\Gamma_f^{\langle t \rangle}$ with values between 0 and 1. This forget gate vector will be multiplied element-wise by the previous cell state $c^{\langle t-1 \rangle}$. So if one of the values of $\Gamma_f^{\langle t \rangle}$ is 0 (or close to 0) then it means that the LSTM should remove that piece of information (e.g. the singular subject) in the corresponding component of $c^{\langle t-1 \rangle}$. If one of the values is 1, then it will keep the information. 

#### - Update gate

Once we forget that the subject being discussed is singular, we need to find a way to update it to reflect that the new subject is now plural. Here is the formulat for the update gate: 

$$\Gamma_u^{\langle t \rangle} = \sigma(W_u[a^{\langle t-1 \rangle}, x^{\{t\}}] + b_u)\tag{2} $$ 

Similar to the forget gate, here $\Gamma_u^{\langle t \rangle}$ is again a vector of values between 0 and 1. This will be multiplied element-wise with $\tilde{c}^{\langle t \rangle}$, in order to compute $c^{\langle t \rangle}$.

#### - Updating the cell 

To update the new subject we need to create a new vector of numbers that we can add to our previous cell state. The equation we use is: 

$$ \tilde{c}^{\langle t \rangle} = \tanh(W_c[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_c)\tag{3} $$

Finally, the new cell state is: 

$$ c^{\langle t \rangle} = \Gamma_f^{\langle t \rangle}* c^{\langle t-1 \rangle} + \Gamma_u^{\langle t \rangle} *\tilde{c}^{\langle t \rangle} \tag{4} $$


#### - Output gate

To decide which outputs we will use, we will use the following two formulas: 

$$ \Gamma_o^{\langle t \rangle}=  \sigma(W_o[a^{\langle t-1 \rangle}, x^{\langle t \rangle}] + b_o)\tag{5}$$ 
$$ a^{\langle t \rangle} = \Gamma_o^{\langle t \rangle}* \tanh(c^{\langle t \rangle})\tag{6} $$

Where in equation 5 you decide what to output using a sigmoid function and in equation 6 you multiply that by the $\tanh$ of the previous state. 

### 2.1 - LSTM cell

**Instructions**:
1. Concatenate $a^{\langle t-1 \rangle}$ and $x^{\langle t \rangle}$ in a single matrix: $concat = \begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}$
2. Compute all the formulas 1-6. You can use `sigmoid()` and `np.tanh()`.
3. Compute the prediction $y^{\langle t \rangle}$. You can use `softmax()`

### 2.2 - Forward pass for LSTM

Now that you have implemented one step of an LSTM, you can now iterate this over this using a for-loop to process a sequence of $T_x$ inputs. 

<img src="https://imgur.com/CFEgAAx.png" style="width:500;height:300px;">
<caption><center> **Figure 5**: LSTM over multiple time-steps. </center></caption>

**Exercise:** Implement `lstm_forward()` to run an LSTM over $T_x$ time-steps. 

**Note**: $c^{\langle 0 \rangle}$ is initialized with zeros.

The forward passes for the basic RNN and the LSTM. When using a deep learning framework, implementing the forward pass is sufficient to build systems that achieve great performance. Now we will see how to do backpropagation in LSTM  and RNNS


## 2.3- LSTM backward pass

### 2.3.1 One Step backward

The LSTM backward pass is slighltly more complicated than the forward one. We have provided you with all the equations for the LSTM backward pass below. (If you enjoy calculus exercises feel free to try deriving these from scratch yourself.) 

### 2.3.2 gate derivatives

$$d \Gamma_o^{\langle t \rangle} = da_{next}*\tanh(c_{next}) * \Gamma_o^{\langle t \rangle}*(1-\Gamma_o^{\langle t \rangle})\tag{7}$$

$$d\tilde c^{\langle t \rangle} = dc_{next}*\Gamma_u^{\langle t \rangle}+ \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * i_t * da_{next} * \tilde c^{\langle t \rangle} * (1-\tanh(\tilde c)^2) \tag{8}$$

$$d\Gamma_u^{\langle t \rangle} = dc_{next}*\tilde c^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * \tilde c^{\langle t \rangle} * da_{next}*\Gamma_u^{\langle t \rangle}*(1-\Gamma_u^{\langle t \rangle})\tag{9}$$

$$d\Gamma_f^{\langle t \rangle} = dc_{next}*\tilde c_{prev} + \Gamma_o^{\langle t \rangle} (1-\tanh(c_{next})^2) * c_{prev} * da_{next}*\Gamma_f^{\langle t \rangle}*(1-\Gamma_f^{\langle t \rangle})\tag{10}$$

### 2.3.3 parameter derivatives 

$$ dW_f = d\Gamma_f^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{11} $$
$$ dW_u = d\Gamma_u^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{12} $$
$$ dW_c = d\tilde c^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{13} $$
$$ dW_o = d\Gamma_o^{\langle t \rangle} * \begin{pmatrix} a_{prev} \\ x_t\end{pmatrix}^T \tag{14}$$

To calculate $db_f, db_u, db_c, db_o$ you just need to sum across the horizontal (axis= 1) axis on $d\Gamma_f^{\langle t \rangle}, d\Gamma_u^{\langle t \rangle}, d\tilde c^{\langle t \rangle}, d\Gamma_o^{\langle t \rangle}$ respectively. Note that you should have the `keep_dims = True` option.

Finally, you will compute the derivative with respect to the previous hidden state, previous memory state, and input.

$$ da_{prev} = W_f^T*d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle}+ W_c^T * d\tilde c^{\langle t \rangle} + W_o^T * d\Gamma_o^{\langle t \rangle} \tag{15}$$
Here, the weights for equations 13 are the first n_a, (i.e. $W_f = W_f[:n_a,:]$ etc...)

$$ dc_{prev} = dc_{next}\Gamma_f^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} * (1- \tanh(c_{next})^2)*\Gamma_f^{\langle t \rangle}*da_{next} \tag{16}$$
$$ dx^{\langle t \rangle} = W_f^T*d\Gamma_f^{\langle t \rangle} + W_u^T * d\Gamma_u^{\langle t \rangle}+ W_c^T * d\tilde c_t + W_o^T * d\Gamma_o^{\langle t \rangle}\tag{17} $$
where the weights for equation 15 are from n_a to the end, (i.e. $W_f = W_f[n_a:,:]$ etc...)


### 3.3 Backward pass through the LSTM RNN

This part is very similar to the `rnn_backward` function you implemented above. You will first create variables of the same dimension as your return variables. You will then iterate over all the time steps starting from the end and call the one step function you implemented for LSTM at each iteration. You will then update the parameters by summing them individually. Finally return a dictionary with the new gradients. 

**Instructions**: Implement the `lstm_backward` function. Create a for loop starting from $T_x$ and going backward. For each step call `lstm_cell_backward` and update the your old gradients by adding the new gradients to them. Note that `dxt` is not updated but is stored.

# Questions and Answers(Q&A)

In LSTM Network (Understanding LSTMs), Why input gate and output gate use tanh? what is the intuition behind this?

A: The reason for using tanh is that its range is between (-1,1) whereas the sigmoid function is (0,1).Actually tanh is extended version of sigmoid.
$$ tanh = 2* \sigma(2x) - 1 $$
So, the gradient of the tanh is almost twice than the sigmoid gradient in the x range (-1.663,1.663) and is almost equal to sigmoid gradient in the x range (-inf,-1.633) U (1.663,inf). So the convergence is faster in the case of tanh because of larger gradients and is more resistant to the vanishing gradient problems.

# Activity Recognition

In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import h5py
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Dense, Dropout
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping
from keras.utils.io_utils import HDF5Matrix

SEQ_LEN = 30
MAX_SEQ_LEN = 200
BATCH_SIZE = 16
EPOCHS = 1000

Using TensorFlow backend.


In [2]:
!git clone https://github.com/mohanrajmit/Human-Action-Classification-.git 

Cloning into 'Human-Action-Classification-'...
remote: Enumerating objects: 1266, done.
remote: Counting objects: 100% (1266/1266), done.
remote: Compressing objects: 100% (1261/1261), done.
remote: Total 1266 (delta 17), reused 1236 (delta 2), pack-reused 0
Receiving objects: 100% (1266/1266), 469.57 MiB | 30.94 MiB/s, done.
Resolving deltas: 100% (17/17), done.
Checking out files: 100% (1218/1218), done.


In [0]:
def main():
    # Get model with pretrained weights.
    base_model = InceptionV3(
    weights='imagenet',
    include_top=True)
    
    
    # We'll extract features at the final pool layer.
    model = Model(
        inputs=base_model.input,
        outputs=base_model.get_layer('avg_pool').output)
    
    # Getting the data
    df = get_data('Human-Action-Classification-/data/data_file.csv')
    
    # Clean the data
    df_clean = clean_data(df)
    
    # Creating index-label maps and inverse_maps
    label_index, index_label = get_class_dict(df_clean)
    
    # Split the dataset into train and test
    train, test = split_train_test(df_clean)
    
    # Encoding the dataset
    encode_dataset(train, model, label_index, "train")
    encode_dataset(test,model,label_index,"test")
    
   

In [0]:
!ls

Human-Action-Classification-  sample_data  test_8.h5  train_8.h5


In [0]:
def get_data(path, if_pd=False):
    """Load our data from file."""
    names = ['partition', 'class', 'video_name', 'frames']
    df = pd.read_csv(path,names=names)
    return df

def get_class_dict(df):
    class_name =  list(df['class'].unique())
    index = np.arange(0, len(class_name))
    label_index = dict(zip(class_name, index))
    index_label = dict(zip(index, class_name))
    return (label_index, index_label)
    
def clean_data(df):
    mask = np.logical_and(df['frames'] >= SEQ_LEN, df['frames'] <= MAX_SEQ_LEN)
    df = df[mask]
    return df
def split_train_test(df):
    partition =  (df.groupby(['partition']))
    un = df['partition'].unique()
    train = partition.get_group(un[0])
    test = partition.get_group(un[1])
    return (train, test)

def preprocess_image(img):
    img = cv2.resize(img, (227,227))
    return preprocess_input(img)
    
    
def encode_video(row, model, label_index):
    cap = cv2.VideoCapture(os.path.join("Human-Action-Classification-/data","UCF-101",str(row["class"].iloc[0]) ,str(row["video_name"].iloc[0]) + ".avi"))
    images = []  
    for i in range(SEQ_LEN):
        ret, frame = cap.read()
        frame = preprocess_image(frame)
        images.append(frame)
    
    
    features = model.predict(np.array(images))
    index = label_index[row["class"].iloc[0]]
    print(index)
    #y_onehot = to_categorical(index, len(label_index.keys()))
    
    return features, index

from keras.utils import np_utils
def encode_dataset(data, model, label_index, phase):
    input_f = []
    output_y = []
    required_classes = ["ApplyEyeMakeup" , "ApplyLipstick" , "Archery" , "BabyCrawling" , "BalanceBeam" ,
                       "BandMarching" , "BaseballPitch" , "Basketball" , "BasketballDunk"]
   
    
    for i in tqdm(range(data.shape[0])):
    # Check whether the given row , is of a class that is required
        if str(data.iloc[[i]]["class"].iloc[0]) in required_classes:
 
            features,y =  encode_video(data.iloc[[i]], model, label_index)
            input_f.append(features)
            output_y.append(y)
        
    
    le_labels = np_utils.to_categorical(output_y)
    f = h5py.File(phase+'_8'+'.h5', 'w')
    f.create_dataset(phase, data=np.array(input_f))
    f.create_dataset(phase+"_labels", data=np.array(le_labels))
    
    del input_f[:]
    del output_y[:]



In [5]:
main()

96116736/96112376 [==============================] - 3s 0us/step


  0%|          | 1/703 [00:04<57:03,  4.88s/it]

0


  0%|          | 2/703 [00:05<40:44,  3.49s/it]

0


...

8


100%|█████████▉| 289/290 [01:07<00:00,  4.28it/s]

8


100%|██████████| 290/290 [01:07<00:00,  4.30it/s]

8


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import LSTM

In [0]:
def lstm():
    """Build a simple LSTM network. We pass the extracted features from
    our CNN to this model predominantly."""
    input_shape = (SEQ_LEN, 2048)
    # Model.
    model = Sequential()
    model.add(LSTM(2048,input_shape=input_shape,dropout=0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(9, activation='softmax'))
    #model.add(Dense(10, activation='softmax'))"""
    checkpoint = ModelCheckpoint(filepath='models\\checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
    
    tb_callback = TensorBoard(
    log_dir="logs",
    histogram_freq=2,
    write_graph=True
    )
    
    early_stopping = EarlyStopping(monitor = 'val_loss',patience= 10)
    
    callback_list = [checkpoint, tb_callback]

    optimizer = Adam(lr=1e-5, decay=1e-6)
    metrics = ['accuracy', 'top_k_categorical_accuracy']
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=metrics)
    return model, callback_list


In [11]:
x_train = HDF5Matrix('train_8.h5', 'train')
y_train = HDF5Matrix('train_8.h5', 'train_labels')
x_test = HDF5Matrix('test_8.h5', 'test')
y_test = HDF5Matrix('test_8.h5', 'test_labels')

print(x_train.shape)
print(y_train.shape)
print(y_train[240])
print(x_test.shape)
print(y_test.shape)
    
model, callback_list = lstm()
#model.fit(x_train, y_train)
model.fit(x_train, y_train, batch_size = BATCH_SIZE, epochs = 100,verbose = 2,validation_data = (x_test, y_test),shuffle = 'batch', callbacks=callback_list)
    
model.save("Activity_Recognition.h5")

(703, 30, 2048)
(703, 9)
[0. 0. 0. 1. 0. 0. 0. 0. 0.]
(290, 30, 2048)
(290, 9)
Train on 703 samples, validate on 290 samples
Epoch 1/100
 - 21s - loss: 2.5590 - acc: 0.0825 - top_k_categorical_accuracy: 0.5050 - val_loss: 2.2605 - val_acc: 0.1621 - val_top_k_categorical_accuracy: 0.5724
Epoch 2/100
 - 18s - loss: 2.4591 - acc: 0.1323 - top_k_categorical_accuracy: 0.5477 - val_loss: 2.2056 - val_acc: 0.2000 - val_top_k_categorical_accuracy: 0.6310
Epoch 3/100
 - 18s - loss: 2.3995 - acc: 0.1110 - top_k_categorical_accuracy: 0.5590 - val_loss: 2.1705 - val_acc: 0.1931 - val_top_k_categorical_accuracy: 0.6552
Epoch 4/100
 - 18s - loss: 2.3812 - acc: 0.1166 - top_k_categorical_accuracy: 0.5861 - val_loss: 2.1540 - val_acc: 0.2207 - val_top_k_categorical_accuracy: 0.6310
Epoch 5/100
 - 18s - loss: 2.3276 - acc: 0.1494 - top_k_categorical_accuracy: 0.6131 - val_loss: 2.1241 - val_acc: 0.2207 - val_top_k_categorical_accuracy: 0.6690
Epoch 6/100
 - 18s - loss: 2.2759 - acc: 0.1607 - top_k_cate

In [12]:
!ls

 Activity_Recognition.h5	   'models\checkpoint-50-1.63.hdf5'
 Human-Action-Classification-	   'models\checkpoint-51-1.61.hdf5'
 logs				   'models\checkpoint-52-1.62.hdf5'
'models\checkpoint-01-2.26.hdf5'   'models\checkpoint-53-1.62.hdf5'
'models\checkpoint-02-2.21.hdf5'   'models\checkpoint-54-1.63.hdf5'
'models\checkpoint-03-2.17.hdf5'   'models\checkpoint-55-1.62.hdf5'
'models\checkpoint-04-2.15.hdf5'   'models\checkpoint-56-1.61.hdf5'
'models\checkpoint-05-2.12.hdf5'   'models\checkpoint-57-1.61.hdf5'
'models\checkpoint-06-2.10.hdf5'   'models\checkpoint-58-1.60.hdf5'
'models\checkpoint-07-2.07.hdf5'   'models\checkpoint-59-1.59.hdf5'
'models\checkpoint-08-2.05.hdf5'   'models\checkpoint-60-1.59.hdf5'
'models\checkpoint-09-2.02.hdf5'   'models\checkpoint-61-1.59.hdf5'
'models\checkpoint-100-1.55.hdf5'  'models\checkpoint-62-1.59.hdf5'
'models\checkpoint-10-2.00.hdf5'   'models\checkpoint-63-1.61.hdf5'
'models\checkpoint-11-1.98.hdf5'   'models\checkpoint-64-1.61.hdf5'
'models\chec

# Sequence to Sequence Models - a general overview

Many times, we might have to convert one sequence to another. Really?  Where?

We do this in machine translation. For this purpose we use models known as sequence to sequence models. (**seq2seq**)

If we take a high-level view, a seq2seq model has encoder, decoder and intermediate step as its main components:
![alt text](https://cdn-images-1.medium.com/max/800/1*3lj8AGqfwEE5KCTJ-dXTvg.png)

A basic sequence-to-sequence model consists of two recurrent neural networks (RNNs): an encoder that processes the input and a decoder that generates the output. This basic architecture is depicted below.
![alt text](https://www.tensorflow.org/images/basic_seq2seq.png)

Each box in the picture above represents a cell of the RNN, most commonly a GRU cell or an LSTM cell. Encoder and decoder can share weights or, as is more common, use a different set of parameters.

In the basic model depicted above, every input has to be encoded into a fixed-size state vector, as that is the only thing passed to the decoder. To allow the decoder more direct access to the input, an **attention** mechanism was introduced. We'll look into details of the attention mechanism in the next part.

## Encoder

Our input sequence is how are you. Each word from the input sequence is associated to a vector 
w∈Rd (via a lookup table). In our case, we have 3 words, thus our input will be transformed into $[w0,w1,w2]∈R^{d×3}$. Then, we simply run an LSTM over this sequence of vectors and store the last hidden state outputed by the LSTM: this will be our encoder representation e. Let’s write the hidden states $[e_0,e_1,e_2]$ (and thus $e=e_2$).
![alt text](https://guillaumegenthial.github.io/assets/img2latex/seq2seq_vanilla_encoder.svg)




## Decoder

Now that we have a vector e that captures the meaning of the input sequence, we’ll use it to generate the target sequence word by word. Feed to another LSTM cell: $e$ as hidden state and a special start of sentence vector $w_{s o s}$ as input. The LSTM computes the next hidden state $h_0 ∈ R h$ . Then, we apply some function $g : R^h ↦ R^V$ so that 
$s_0 := g ( h_0 ) ∈ R^V$ is a vector of the same size as the vocabulary.

\begin{equation}
h_0 = LSTM ( e , w_{s o s} ) 
\end{equation}
\begin{equation}
s_0 = g ( h_0 )
\end{equation}
\begin{equation}
p_0 = softmax ( s_0 )
\end{equation}
\begin{equation}
i_0 = argmax ( p_0 )$
\end{equation}

Then, apply a softmax to $s_ 0$ to normalize it into a vector of probabilities $p_0 ∈ R^V$ . Now, each entry of $p_0$ will measure how likely is each word in the vocabulary. Let’s say that the word “comment” has the highest probability (and thus $i_0 = argmax ( p_0 )$ corresponds to the index of “comment”). Get a corresponding vector $w_{i_0} = w_{comment}$ and repeat the procedure: the LSTM will take $h_0$ as hidden state and $w_{comment}$ as input and will output a probability vector $p_1$ over the second word, etc.

\begin{equation}
h_1 = LSTM ( h_0 , w_{i_0} ) 
\end{equation}
\begin{equation}
s_1 = g ( h_1 )
\end{equation}
\begin{equation}
p_1 = softmax ( s_1 ) 
\end{equation}
\begin{equation}
i _1 = argmax ( p_1 )
\end{equation}

The decoding stops when the predicted word is a special end of sentence token.

![alt text](https://guillaumegenthial.github.io/assets/img2latex/seq2seq_vanilla_decoder.svg)

# Attention !!!

![alt text](https://guillaumegenthial.github.io/assets/img2latex/seq2seq_attention_mechanism_new.svg)

## Seq2Seq with Attention 

The previous model has been refined over the past few years and greatly benefited from what is known as attention. Attention is a mechanism that forces the model to learn to focus (= to attend) on specific parts of the input sequence when decoding, instead of relying only on the hidden vector of the decoder’s LSTM. One way of performing attention is as follows. We slightly modify the reccurrence formula that we defined above by adding a new vector $c_t$ to the input of the LSTM
\begin{equation}
h_t = LSTM ( h_{t − 1} , [ w_{i_{t − 1 }}, c_t ] )
\end{equation}
\begin{equation}
s_t = g ( h_t )
\end{equation}
\begin{equation}
 p_t = softmax ( s_t )
\end{equation}
\begin{equation}
 i_t = argmax ( p_t )
\end{equation}

 The vector c_t is the attention (or context) vector. We compute a new context vector at each decoding step. First, with a function $f ( h_{t − 1} , e_{t ′} ) ↦ α t ′ ∈ R$ , compute a score for each hidden state $e_{t'}$ of the encoder. Then, normalize the sequence of $αt′$ using a softmax and compute c t as the weighted average of the $e_{t ′}$.

$α_t ′ = f ( h_{t − 1 }, e_{t ′} ) ∈ R$                 for all $t ′$  

$\vec{\alpha} = softmax ( α ) $

$c_t = n \sum_{t'=0}^{n}  \vec{\alpha}_{t′} e_{t ′}$

The choice of the function  $f$ varies